In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
# import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras import Model, layers
from tensorflow.keras.models import load_model, model_from_json
from tensorflow.keras.layers import GlobalAveragePooling2D
import tensorflow as tf
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
print(f'Keras Version is {tf.keras.__version__}')
print(f'Tensorflow Version is {tf.__version__}')

In [ ]:
data_path = './Data/NEUData/'
BATCH_SIZE = 32
IMG_SIZE = (224,224)

In [ ]:
train_datagen = ImageDataGenerator(shear_range=10
                                   ,zoom_range=0.2
                                   ,horizontal_flip=True
                                   ,preprocessing_function=preprocess_input)

In [ ]:
train_generator = train_datagen.flow_from_directory(data_path+'Train'
                                                    ,batch_size=BATCH_SIZE
                                                    ,class_mode='categorical'
                                                    ,target_size=IMG_SIZE)

In [ ]:
# validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
# validation_generator = validation_datagen.flow_from_directory(data_path+'validation'
#                                                               ,shuffle=False
#                                                               ,class_mode='categorical'
#                                                               ,target_size=IMG_SIZE)

In [ ]:
base_model = ResNet50(include_top=False,weights='imagenet')

In [ ]:
for layers in base_model.layers:
    layers.trainable = False
    print(layers)

In [ ]:
from tensorflow.keras import Model, layers
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024,activation='relu')(x)
x = layers.Dense(512,activation='relu')(x)
predictions = layers.Dense(6, activation='softmax')(x)
model = Model(base_model.input, predictions)

In [ ]:

optimizer = tf.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=10,
                              epochs=5
                             )

In [ ]:
plt.plot(history.history['accuracy'],label='Train Accuracy')
plt.plot(history.history['loss'],label='Train Loss')
plt.plot(history.history['val_accuracy'],label='Validation Accuracy')
plt.legend()

In [ ]:
predict = model.predict_generator(validation_generator)

In [ ]:
validation_generator.labels

In [ ]:
y_pred = []
for y in predict:
    y_pred.append(np.argmax(y))

In [ ]:
confusionMatrix = confusion_matrix(validation_generator.labels,y_pred)
print(confusionMatrix)

In [ ]:
label = np.array(['Crazing', 'Inclusion', 'Patches', 'Pitted Surface', 'Rolled In Scale', 'Scratches'])
labels = np.array([' ','Cr', 'In', 'Pa', 'PS', 'RS', 'Sc'])
print(classification_report(validation_generator.labels,y_pred,target_names=label))

In [ ]:
fig,ax = plt.subplots(figsize=(10,8))
im = ax.imshow(confusionMatrix, cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)
ax.set_xlabel("Actual Label",fontsize=16)
ax.set_ylabel("Predicted Label",fontsize=16)
ax.set_xticklabels(labels,fontsize=18)
ax.set_yticklabels(labels,fontsize=18)
thresh = confusionMatrix.max() / 4.
for i in range(confusionMatrix.shape[0]):
        for j in range(confusionMatrix.shape[1]):
            ax.text(j, i, format(confusionMatrix[i, j], ''),fontsize="14",
                    ha="center", va="center",
                    color="white" if confusionMatrix[i, j] > thresh else "black")
fig.tight_layout(h_pad=0,w_pad=0)
plt.show()